In [6]:
import pandas as pd

data_root = './data/'
train_data=pd.read_csv(data_root+'train.tsv',sep='\t')
test_data=pd.read_csv(data_root+'test.tsv',sep='\t')
pd.set_option('display.width', 900)
print(train_data)

Phrases = train_data['Phrase']
MaxSeqlen = 0
for p in Phrases:
    words = p.split()
    if len(words) > MaxSeqlen:
        MaxSeqlen = len(words)
MaxSeqlen

PhraseId  SentenceId                                             Phrase  Sentiment
0              1           1  A series of escapades demonstrating the adage ...          1
1              2           1  A series of escapades demonstrating the adage ...          2
2              3           1                                           A series          2
3              4           1                                                  A          2
4              5           1                                             series          2
...          ...         ...                                                ...        ...
156055    156056        8544                                          Hearst 's          2
156056    156057        8544                          forced avuncular chortles          1
156057    156058        8544                                 avuncular chortles          3
156058    156059        8544                                          avuncular          2
156059 

52

In [9]:
import numpy as np

word2id = {}
# get word-index dictionary
for i,p in enumerate(Phrases):
    words = p.split()
    for word in words:
        if word not in word2id:
            word2id[word]=len(word2id)

BOW = np.zeros((len(Phrases),len(word2id)), dtype=np.float32)

for i,p in enumerate(Phrases):
    words = p.split()
    for word in words:
        BOW[i][word2id[word]] += 1

print(BOW[:20])
labels = train_data['Sentiment']
label_list = list(set(labels))

0    A series of escapades demonstrating the adage ...
1    A series of escapades demonstrating the adage ...
2                                             A series
3                                                    A
4                                               series
Name: Phrase, dtype: object


MemoryError: Unable to allocate 21.2 GiB for an array with shape (156060, 18226) and data type float64

## Softmax regression

In [8]:
n_iter = 10
batch_size = 128# 
n_features =len(word2id)n
n_class = len(label_list)
lr = 1e-4
print_every = 500

def make_network(n_hidden=128):
    model = dict(
        W1=np.random.randnword2idn, n_hidden),
        W2=np.random.randn(n_hidden, n_class)
    )
    return model

def softmax(x):
    return np.exp(x)/(np.sum(np.exp(x),axis=1)[:,None])

def forward(x,w):
    h = x @ w['W1']
   #  h[h<0] = 0
    x = h @ w['W2']
    output = softmax(x)
    return h, output

def ce_loss(probs, labels):
    log_probs = np.log(probs)
    return -np.sum(log_probs[:,labels])

def backward(train_data, h, w, output):
    err = output - 1
    dW2 = h.T @ err
    dh = err @ w['W2'].T
   #  dh[h<0] = 0
    dW1 = train_data.T @ dh
    return dict(W1=dW1, W2=dW2)

def sgd_step(w, grad_w):
    for param in w:
        w[param] -= lr*grad_w[param]

model = make_network()
step = 0
for i in range(n_iter):
    start_idx = 0
    np.random.shuffle(BOW)
    while start_idx + batch_size <= BOW.shape[0]:
        train_batch = BOW[start_idx:start_idx+batch_size]
        train_label = labels[start_idx:start_idx+batch_size]
        h, output = forward(train_batch, model)
        loss = ce_loss(output, train_label)
        model_grad = backward(train_batch, h, model, output)
        print(model['W1'][0][:10], model_grad['W1'][0][:10])
        sgd_step(model, model_grad)
        if step%print_every == 0:
            print("Step %d, loss: %.2f"%(step,loss))
        step += 1
        start_idx += batch_size

SyntaxError: invalid syntax (<ipython-input-8-94a03cb497cc>, line 3)